In [ ]:
cd /Users/juiashinkar/Recommenders

/Users/juiashinkar/Recommenders


In [ ]:
import tensorflow as tf
tf.__version__

'1.15.3'

In [ ]:
import sys
sys.path.append("../../")
import os
import logging
import papermill as pm
from tempfile import TemporaryDirectory

import tensorflow as tf
import time

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    prepare_hparams
)
from reco_utils.dataset.amazon_reviews import download_and_extract, data_preprocessing
from reco_utils.dataset.download_utils import maybe_download
#from reco_utils.recommender.deeprec.models.sequential.sli_rec import SLI_RECModel
####  to use the other model, use one of the following lines:
from reco_utils.recommender.deeprec.models.sequential.asvd import A2SVDModel
#from reco_utils.recommender.deeprec.models.sequential.caser import CaserModel
#from reco_utils.recommender.deeprec.models.sequential.gru4rec import GRU4RecModel
#from reco_utils.recommender.deeprec.models.sequential.nextitnet import NextItNetModel

from reco_utils.recommender.deeprec.io.sequential_iterator import SequentialIterator
#from reco_utils.recommender.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.6.10 |Anaconda, Inc.| (default, May  7 2020, 23:06:31) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Tensorflow version: 1.15.3


In [ ]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result
yaml_file = '/Users/juiashinkar/Recommenders/reco_utils/recommender/deeprec/config/sli_rec.yaml'
data_path = '/Users/juiashinkar/Recommenders/tests/resources/deeprec/slirec'

In [ ]:
# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = '/Users/juiashinkar/Desktop/reviews_Movies_and_TV_5.json'
meta_name = '/Users/juiashinkar/Desktop/meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
sample_rate = 0.01 # sample a small item set for training and testing here for fast example

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

In [ ]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001, 
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
            )

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:

input_creator = SequentialIterator
#### uncomment this for the NextItNet model, because it needs a special data iterator for training
#input_creator = NextItNetIterator

In [ ]:

model = A2SVDModel(hparams, input_creator, seed=RANDOM_SEED)
##  of course you can create models like ASVDModel, CaserModel and GRU4RecModel in the same manner


## sometimes we don't want to train a model from scratch
## then we can load a pre-trained model like this: 
#model.load_model(r'your_model_path')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where











In [ ]:
print(model.run_eval(test_file, num_ngs=test_num_ngs)) # test_num_ngs is the number of negative lines after each positive line in your test_file

{'auc': 0.4909, 'logloss': 0.6931, 'mean_mrr': 0.2737, 'ndcg@2': 0.1423, 'ndcg@4': 0.2284, 'ndcg@6': 0.3017, 'group_auc': 0.4907}


In [ ]:
start_time = time.time()
model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 
# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
end_time = time.time()
print('Time cost for training is {0:.2f} mins'.format((end_time-start_time)/60.0))


step 20 , total_loss: 1.6134, data_loss: 1.6134
step 40 , total_loss: 1.6087, data_loss: 1.6087
eval valid at epoch 1: auc:0.4959,logloss:0.6931,mean_mrr:0.4501,ndcg@2:0.3171,ndcg@4:0.5049,ndcg@6:0.5846,group_auc:0.4929
step 20 , total_loss: 1.6065, data_loss: 1.6065
step 40 , total_loss: 1.5913, data_loss: 1.5913
eval valid at epoch 2: auc:0.4952,logloss:0.697,mean_mrr:0.4513,ndcg@2:0.3171,ndcg@4:0.5058,ndcg@6:0.5856,group_auc:0.4932
step 20 , total_loss: 1.5271, data_loss: 1.5271
step 40 , total_loss: 1.4993, data_loss: 1.4993
eval valid at epoch 3: auc:0.5934,logloss:0.7245,mean_mrr:0.5253,ndcg@2:0.4248,ndcg@4:0.5901,ndcg@6:0.6425,group_auc:0.5877
step 20 , total_loss: 1.4171, data_loss: 1.4171
step 40 , total_loss: 1.3922, data_loss: 1.3922
eval valid at epoch 4: auc:0.6904,logloss:0.641,mean_mrr:0.6118,ndcg@2:0.5443,ndcg@4:0.6781,ndcg@6:0.7086,group_auc:0.6887
step 20 , total_loss: 1.3244, data_loss: 1.3244
step 40 , total_loss: 1.4050, data_loss: 1.4050
eval valid at epoch 5: au

In [ ]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)
pm.record("res_syn", res_syn)

{'auc': 0.6938, 'logloss': 0.7222, 'mean_mrr': 0.4561, 'ndcg@2': 0.3584, 'ndcg@4': 0.4702, 'ndcg@6': 0.5253, 'group_auc': 0.6874}


/Applications/anaconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model = model.predict(test_file, output_file)